In [1]:
comp2id = {}
with open("raw/vocab_compound") as f:
    for index, line in enumerate(f):
        comp2id[line.strip()] = index
print(comp2id)

{'_PAD': 0, '_GO': 1, '_EOS': 2, '_UNK': 3, 'C': 4, '=': 5, '(': 6, ')': 7, 'O': 8, 'N': 9, '1': 10, '2': 11, '3': 12, '4': 13, '[': 14, ']': 15, 'S': 16, 'l': 17, 'F': 18, '-': 19, '5': 20, '+': 21, '.': 22, '6': 23, 'B': 24, 'r': 25, '#': 26, 'P': 27, 'i': 28, 'H': 29, '7': 30, 'I': 31, '8': 32, '9': 33, 'a': 34, 'e': 35, 'A': 36, 'n': 37, 's': 38, 'u': 39, 'g': 40, 'o': 41, 't': 42, 'T': 43, 'M': 44, 'Z': 45, 'b': 46, 'K': 47, 'R': 48, 'd': 49, 'W': 50, 'G': 51, 'L': 52, 'c': 53, 'h': 54, 'V': 55, 'm': 56, 'E': 57, 'Y': 58, 'U': 59, 'f': 60, 'D': 61, 'y': 62, '%': 63, '0': 64, 'p': 65, 'k': 66, 'X': 67}


In [2]:
import torch

def insert_padding(mylist, max_size):
    for i, sentence in enumerate(mylist):
        templist = [0 for _ in range(max_size-len(sentence))]
        mylist[i] = mylist[i] + templist
    return mylist

def process_independent(mylist, max_size, file_name):
    mylist = insert_padding(mylist, max_size)
    mytensor = torch.tensor(mylist)
    torch.save(mytensor, 'data2/'+file_name+".pt")
    return mytensor

comp_MAX_size = 100

In [3]:
from tqdm import tqdm
import numpy as np
import gc

for file_num in range(6):  # 0부터 5까지 파일 번호
    file_path = f'intermediate/chunk_{file_num}.tsv'
    smiles_ids = []  # 현재 파일의 결과를 저장할 리스트

    with open(file_path, 'r') as f:
        for line in tqdm(f, desc=f"Processing chunk {file_num}"):
            sentence = line.strip().split('\t')[0]  # 수정: SMILES 문자열만 추출
            temp = []
            skip_string = False
            for char in sentence:
                if char not in comp2id:
                    skip_string = True
                    break
                else:
                    temp.append(comp2id[char])
            if not skip_string and len(temp) <= comp_MAX_size:  # 수정: 길이 조건 추가
                smiles_ids.append(temp)

    process_independent(smiles_ids, comp_MAX_size, f"processed/smiles_ids_{file_num}")

    # 파일 처리 후 메모리 관리
    smiles_ids = []
    gc.collect()  # 파일 하나 처리 후 강제 가비지 컬렉션


Processing chunk 0: 23244871it [01:15, 306571.85it/s]


: 